## Predicting Survival on the Titanic

### History
Perhaps one of the most infamous shipwrecks in history, the Titanic sank after colliding with an iceberg, killing 1502 out of 2224 people on board. Interestingly, by analysing the probability of survival based on few attributes like gender, age, and social status, we can make very accurate predictions on which passengers would survive. Some groups of people were more likely to survive than others, such as women, children, and the upper-class. Therefore, we can learn about the society priorities and privileges at the time.

### Assignment:

Build a Machine Learning Pipeline, to engineer the features in the data set and predict who is more likely to Survive the catastrophe.

Follow the Jupyter notebook below, and complete the missing bits of code, to achieve each one of the pipeline steps.

In [712]:
import re

# to handle datasets
import pandas as pd
import numpy as np

# for visualization
import matplotlib.pyplot as plt

# to divide train and test set
from sklearn.model_selection import train_test_split

# feature scaling
from sklearn.preprocessing import StandardScaler

# to build the models
from sklearn.linear_model import LogisticRegression

# to evaluate the models
from sklearn.metrics import accuracy_score, roc_auc_score

# to persist the model and the scaler
import joblib

# to visualise al the columns in the dataframe
pd.pandas.set_option('display.max_columns', None)

## Prepare the data set

In [713]:
# load the data - it is available open source and online

data = pd.read_csv('https://www.openml.org/data/get_csv/16826755/phpMYEkMl')

# display data
data.head()

,pclass,survived,name,sex,age,sibsp,parch,ticket,fare,cabin,embarked,boat,body,home.dest
0,1,1,"Allen, Miss. Elisabeth Walton",female,29,0,0,24160,211.3375,B5,S,2,?,"St Louis, MO"
1,1,1,"Allison, Master. Hudson Trevor",male,0.9167,1,2,113781,151.55,C22 C26,S,11,?,"Montreal, PQ / Chesterville, ON"
2,1,0,"Allison, Miss. Helen Loraine",female,2,1,2,113781,151.55,C22 C26,S,?,?,"Montreal, PQ / Chesterville, ON"
3,1,0,"Allison, Mr. Hudson Joshua Creighton",male,30,1,2,113781,151.55,C22 C26,S,?,135,"Montreal, PQ / Chesterville, ON"
4,1,0,"Allison, Mrs. Hudson J C (Bessie Waldo Daniels)",female,25,1,2,113781,151.55,C22 C26,S,?,?,"Montreal, PQ / Chesterville, ON"


In [714]:
# replace interrogation marks by NaN values

data = data.replace('?', np.nan)

In [715]:
# retain only the first cabin if more than
# 1 are available per passenger

def get_first_cabin(row):
    try:
        return row.split()[0]
    except:
        return np.nan

data['cabin'] = data['cabin'].apply(get_first_cabin)

In [716]:
# extracts the title (Mr, Ms, etc) from the name variable

def get_title(passenger):
    line = passenger
    if re.search('Mrs', line):
        return 'Mrs'
    elif re.search('Mr', line):
        return 'Mr'
    elif re.search('Miss', line):
        return 'Miss'
    elif re.search('Master', line):
        return 'Master'
    else:
        return 'Other'

data['title'] = data['name'].apply(get_title)

In [717]:
# cast numerical variables as floats

data['fare'] = data['fare'].astype('float')
data['age'] = data['age'].astype('float')

In [718]:
# drop unnecessary variables

data.drop(labels=['name','ticket', 'boat', 'body','home.dest'], axis=1, inplace=True)

# display data
data.head()

,pclass,survived,sex,age,sibsp,parch,fare,cabin,embarked,title
0,1,1,female,29.0000,0,0,211.3375,B5,S,Miss
1,1,1,male,0.9167,1,2,151.5500,C22,S,Master
2,1,0,female,2.0000,1,2,151.5500,C22,S,Miss
3,1,0,male,30.0000,1,2,151.5500,C22,S,Mr
4,1,0,female,25.0000,1,2,151.5500,C22,S,Mrs


In [719]:
# save the data set

data.to_csv('titanic.csv', index=False)

## Data Exploration

### Find numerical and categorical variables

In [720]:
target = 'survived'

In [721]:
all_cols = data.columns
vars_num = ['pclass', 'sibsp', 'parch', 'age', 'fare']

vars_cat = [col for col in all_cols if col not in vars_num + [target]]

print('Number of numerical variables: {}'.format(len(vars_num)))
print('Number of categorical variables: {}'.format(len(vars_cat)))

Number of numerical variables: 5
Number of categorical variables: 4


In [722]:
vars_cat

['sex', 'cabin', 'embarked', 'title']

In [723]:
vars_num

['pclass', 'sibsp', 'parch', 'age', 'fare']

### Find missing values in variables

In [724]:
# first in numerical variables

missing_id = data[vars_num].isna()

In [725]:
# now in categorical variables



### Determine cardinality of categorical variables

### Determine the distribution of numerical variables

## Separate data into train and test

Use the code below for reproducibility. Don't change it.

In [726]:
X_train, X_test, y_train, y_test = train_test_split(
    data.drop('survived', axis=1),  # predictors
    data['survived'],  # target
    test_size=0.2,  # percentage of obs in test set
    random_state=0)  # seed to ensure reproducibility

X_train.shape, X_test.shape

((1047, 9), (262, 9))

## Feature Engineering

### Extract only the letter (and drop the number) from the variable Cabin

In [727]:

X_train['cabin'] = X_train['cabin'].str[0]
X_test['cabin'] = X_test['cabin'].str[0]

In [728]:
X_train

,pclass,sex,age,sibsp,parch,fare,cabin,embarked,title
1118,3,male,25.0000,0,0,7.9250,NaN,S,Mr
44,1,female,41.0000,0,0,134.5000,E,C,Miss
1072,3,male,NaN,0,0,7.7333,NaN,Q,Mr
1130,3,female,18.0000,0,0,7.7750,NaN,S,Miss
574,2,male,29.0000,1,0,21.0000,NaN,S,Mr
...,...,...,...,...,...,...,...,...,...
763,3,female,0.1667,1,2,20.5750,NaN,S,Miss
835,3,male,NaN,0,0,8.0500,NaN,S,Mr
1216,3,female,NaN,0,0,7.7333,NaN,Q,Miss
559,2,female,20.0000,0,0,36.7500,NaN,S,Miss


### Fill in Missing data in numerical variables:

- Add a binary missing indicator
- Fill NA in original variable with the median

In [729]:
for col in ['age', 'fare']:
    X_train[col + '_missing'] = X_train[col].isna().astype(int)
    X_test[col + '_missing'] = X_test[col].isna().astype(int)
    med = X_train[col].median()
    
    X_train[col] = X_train[col].fillna(med)
    X_test[col] = X_test[col].fillna(med)

In [730]:
X_train

,pclass,sex,age,sibsp,parch,fare,cabin,embarked,title,age_missing,fare_missing
1118,3,male,25.0000,0,0,7.9250,NaN,S,Mr,0,0
44,1,female,41.0000,0,0,134.5000,E,C,Miss,0,0
1072,3,male,28.0000,0,0,7.7333,NaN,Q,Mr,1,0
1130,3,female,18.0000,0,0,7.7750,NaN,S,Miss,0,0
574,2,male,29.0000,1,0,21.0000,NaN,S,Mr,0,0
...,...,...,...,...,...,...,...,...,...,...,...
763,3,female,0.1667,1,2,20.5750,NaN,S,Miss,0,0
835,3,male,28.0000,0,0,8.0500,NaN,S,Mr,1,0
1216,3,female,28.0000,0,0,7.7333,NaN,Q,Miss,1,0
559,2,female,20.0000,0,0,36.7500,NaN,S,Miss,0,0


### Replace Missing data in categorical variables with the string **Missing**

In [731]:
for col in vars_cat:
    X_train[col] = X_train[col].fillna('Missing')
    X_test[col] = X_test[col].fillna('Missing')

### Remove rare labels in categorical variables

- remove labels present in less than 5 % of the passengers

In [732]:
X_train.isnull().sum()

pclass          0
sex             0
age             0
sibsp           0
parch           0
fare            0
cabin           0
embarked        0
title           0
age_missing     0
fare_missing    0
dtype: int64

In [733]:
n = len(X_train)
thres = 0.05

In [734]:
rare_labels = []
for col in vars_cat:
    rare_labels = X_train[col].value_counts()[X_train[col].value_counts()/n < thres].index
    
    X_train[col] = X_train[col].replace(rare_labels, 'Rare')
    X_test[col] = X_test[col].replace(rare_labels, 'Rare')


In [735]:
X_train

,pclass,sex,age,sibsp,parch,fare,cabin,embarked,title,age_missing,fare_missing
1118,3,male,25.0000,0,0,7.9250,Missing,S,Mr,0,0
44,1,female,41.0000,0,0,134.5000,Rare,C,Miss,0,0
1072,3,male,28.0000,0,0,7.7333,Missing,Q,Mr,1,0
1130,3,female,18.0000,0,0,7.7750,Missing,S,Miss,0,0
574,2,male,29.0000,1,0,21.0000,Missing,S,Mr,0,0
...,...,...,...,...,...,...,...,...,...,...,...
763,3,female,0.1667,1,2,20.5750,Missing,S,Miss,0,0
835,3,male,28.0000,0,0,8.0500,Missing,S,Mr,1,0
1216,3,female,28.0000,0,0,7.7333,Missing,Q,Miss,1,0
559,2,female,20.0000,0,0,36.7500,Missing,S,Miss,0,0


### Perform one hot encoding of categorical variables into k-1 binary variables

- k-1, means that if the variable contains 9 different categories, we create 8 different binary variables
- Remember to drop the original categorical variable (the one with the strings) after the encoding

In [678]:
vars_cat

['sex', 'cabin', 'embarked', 'title']

In [679]:
# pd.get_dummies(X_test['title'], drop_first=True, prefix='title')
X_test['title'].nunique()

4

In [736]:
import pandas as pd
for col in vars_cat:
    X_train = pd.concat([X_train.drop(columns=col), pd.get_dummies(X_train[col], drop_first=True, prefix=col)], axis=1)
    X_test = pd.concat([X_test.drop(columns=col), pd.get_dummies(X_test[col], drop_first=True, prefix=col)], axis=1)


In [740]:
diff_col = list(set(X_train.columns) - set(X_test.columns))

In [742]:
X_test[diff_col] = False

In [746]:
X_test = X_test[[col for col in X_train.columns]]

In [351]:
# False since we know that the new columns in test are false for all (otherwise a dummy would have already existed)
# X_test = X_test.reindex(columns=X_train.columns, fill_value=False)

### Scale the variables

- Use the standard scaler from Scikit-learn

In [747]:
X_test

,pclass,age,sibsp,parch,fare,age_missing,fare_missing,sex_male,cabin_Missing,cabin_Rare,embarked_Q,embarked_Rare,embarked_S,title_Mr,title_Mrs,title_Rare
1139,3,38.0,0,0,7.8958,0,0,True,True,False,False,False,True,True,False,False
533,2,21.0,0,1,21.0000,0,0,False,True,False,False,False,True,False,False,False
459,2,42.0,1,0,27.0000,0,0,True,True,False,False,False,True,True,False,False
1150,3,28.0,0,0,14.5000,1,0,True,True,False,False,False,True,True,False,False
393,2,25.0,0,0,31.5000,0,0,True,True,False,False,False,True,True,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
753,3,22.0,0,0,8.0500,0,0,True,True,False,False,False,True,True,False,False
1052,3,28.0,0,0,7.8958,1,0,True,True,False,False,False,True,True,False,False
426,2,30.0,0,0,13.0000,0,0,True,True,False,False,False,True,True,False,False
554,2,26.0,0,0,13.0000,0,0,True,True,False,False,False,True,True,False,False


In [751]:
scaler = StandardScaler()

# Fit and transform the data
X_train = scaler.fit_transform(X_train)
X_test = scaler.fit_transform(X_test)
# X_test

In [760]:
# X_train[:,12:15]

array([[ 0.63964997,  0.86989161, -0.43562912],
       [-1.56335503, -1.14956851, -0.43562912],
       [-1.56335503,  0.86989161, -0.43562912],
       ...,
       [-1.56335503, -1.14956851, -0.43562912],
       [ 0.63964997, -1.14956851, -0.43562912],
       [-1.56335503, -1.14956851,  2.29553067]])

## Train the Logistic Regression model

- Set the regularization parameter to 0.0005
- Set the seed to 0

In [752]:
lr = LogisticRegression(C=0.0005, random_state=0)
mod = lr.fit(X_train, y_train)

y_pred_probs = mod.predict_proba(X_test)

## Make predictions and evaluate model performance

Determine:
- roc-auc
- accuracy

**Important, remember that to determine the accuracy, you need the outcome 0, 1, referring to survived or not. But to determine the roc-auc you need the probability of survival.**

In [753]:
auc = roc_auc_score(y_test, y_pred_probs[:,1])

y_pred = np.argmax(y_pred_probs, axis=1)
accuracy = accuracy_score(y_test, y_pred)


print(f'AUC: {auc:.2f}')
print(f'Accuracy: {accuracy:.2f}')


AUC: 0.84
Accuracy: 0.71


That's it! Well done

**Keep this code safe, as we will use this notebook later on, to build production code, in our next assignement!!**